Importing the Dependencies

In [19]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn import svm
from sklearn.metrics import accuracy_score
from sklearn.metrics import (confusion_matrix, matthews_corrcoef, 
                             brier_score_loss, accuracy_score, roc_auc_score)

Data Collection & Analysis

In [2]:
# loading the data from csv file to a Pandas DataFrame
parkinsons_data = pd.read_csv('/Users/Siddhesh/My Files/VIT/PFE/PFECP/Data/parkinsons.csv')

In [3]:
# grouping the data based on the target variable
parkinsons_data.drop('name', axis=1).groupby('status').mean()

,MDVP:Fo(Hz),MDVP:Fhi(Hz),MDVP:Flo(Hz),MDVP:Jitter(%),MDVP:Jitter(Abs),MDVP:RAP,MDVP:PPQ,Jitter:DDP,MDVP:Shimmer,MDVP:Shimmer(dB),...,MDVP:APQ,Shimmer:DDA,NHR,HNR,RPDE,DFA,spread1,spread2,D2,PPE
status,,,,,,,,,,,,,,,,,,,,,
0,181.937771,223.636750,145.207292,0.003866,0.000023,0.001925,0.002056,0.005776,0.017615,0.162958,...,0.013305,0.028511,0.011483,24.678750,0.442552,0.695716,-6.759264,0.160292,2.154491,0.123017
1,145.180762,188.441463,113.016007,0.006989,0.000051,0.003757,0.003900,0.011273,0.033658,0.321204,...,0.027600,0.053027,0.029211,20.974048,0.516816,0.725408,-5.333420,0.248133,2.456058,0.233828


In [4]:
X = parkinsons_data.drop(columns=['name','status'], axis=1)
Y = parkinsons_data['status']

In [5]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=2)

In [6]:
print(X.shape, X_train.shape, X_test.shape)

(195, 22) (156, 22) (39, 22)


Model Training

Support Vector Machine Model

In [7]:
model = svm.SVC(kernel='linear')
# training the SVM model with training data
model.fit(X_train, Y_train)

SVC(kernel='linear')

Model Evaluation

In [8]:
X_train_prediction = model.predict(X_train)
training_data_accuracy = accuracy_score(X_train_prediction, Y_train)

from sklearn.calibration import CalibratedClassifierCV

calibrated_svm = CalibratedClassifierCV(model, method="sigmoid")
calibrated_svm.fit(X_train, Y_train)

# Predictions
y_pred = model.predict(X_test)
y_prob = calibrated_svm.predict_proba(X_test)[:, 1]  # Probability estimates for Brier Score & AUC

Accuracy Score

In [9]:
# accuracy score on training data
X_train_prediction = model.predict(X_train)
training_data_accuracy = accuracy_score(Y_train, X_train_prediction)

In [10]:
print('Accuracy score of training data : ', training_data_accuracy)

Accuracy score of training data :  0.8717948717948718


In [11]:
# accuracy score on training data
X_test_prediction = model.predict(X_test)
test_data_accuracy = accuracy_score(Y_test, X_test_prediction)

In [12]:
print('Accuracy score of test data : ', test_data_accuracy)

Accuracy score of test data :  0.8717948717948718


Building a Predictive System

In [13]:
input_data = (197.07600,206.89600,192.05500,0.00289,0.00001,0.00166,0.00168,0.00498,0.01098,0.09700,0.00563,0.00680,0.00802,0.01689,0.00339,26.77500,0.422229,0.741367,-7.348300,0.177551,1.743867,0.085569)

# changing input data to a numpy array
input_data_as_numpy_array = np.asarray(input_data)

# reshape the numpy array
input_data_reshaped = input_data_as_numpy_array.reshape(1,-1)

prediction = model.predict(input_data_reshaped)
print(prediction)


if (prediction[0] == 0):
  print("The Person does not have Parkinsons Disease")

else:
  print("The Person has Parkinsons")


[0]
The Person does not have Parkinsons Disease


/opt/anaconda3/lib/python3.12/site-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but SVC was fitted with feature names
  warnings.warn(


Saving the trained model

In [15]:
import pickle
filename = 'parkinsons_svm.sav'
pickle.dump(model, open(filename, 'wb'))
# loading the saved model
loaded_model = pickle.load(open('parkinsons_svm.sav', 'rb'))

In [16]:
from sklearn.metrics import classification_report
report = classification_report(Y_test, X_test_prediction)
print(report)

              precision    recall  f1-score   support

           0       0.80      0.50      0.62         8
           1       0.88      0.97      0.92        31

    accuracy                           0.87        39
   macro avg       0.84      0.73      0.77        39
weighted avg       0.87      0.87      0.86        39



In [17]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(Y_test, y_pred)
print("Confusion Matrix:\n", cm)

# Compute confusion matrix values
tn, fp, fn, tp = confusion_matrix(Y_test, y_pred).ravel()

Confusion Matrix:
 [[ 4  4]
 [ 1 30]]


In [20]:

# Compute required metrics
tpr = tp / (tp + fn)  # Sensitivity / Recall
tnr = tn / (tn + fp)  # Specificity
mcc = matthews_corrcoef(Y_test, y_pred)  # MCC
brier = brier_score_loss(Y_test, y_prob)  # Brier Score
accuracy = accuracy_score(Y_test, y_pred)  # Accuracy
auc = roc_auc_score(Y_test, y_prob)  # AUC-ROC

In [21]:

# Print results
print(f"TPR (Sensitivity): {tpr:.4f}")
print(f"TNR (Specificity): {tnr:.4f}")
print(f"MCC: {mcc:.4f}")
print(f"Brier Score: {brier:.4f}")
print(f"Accuracy: {accuracy:.4f}")
print(f"AUC-ROC: {auc:.4f}")

TPR (Sensitivity): 0.9677
TNR (Specificity): 0.5000
MCC: 0.5649
Brier Score: 0.1166
Accuracy: 0.8718
AUC-ROC: 0.8347
